Esta práctica está compuesta por tres apartados, en los que se deben localizar y reconocer caracteres de matrículas de
coches.

# Apartado 1: Localización de los caracteres de la matrícula

Este apartado aparece desarrollado en los ficheros *localizacion_caracteres.py*, *localizacion_matricula* y 
*deteccion_haar.py*.

En el fichero *deteccion_haar.py* se desarrolla la clase *HaarDetector*, que recibe un fichero XML para entrenar un 
clasificador en cascada que servirá para localizar la posición de la matricula de los coches.

Se importa la libreria a utilizar, OpenCV, y se define la clase *HaarDetector*. El contructor de esta clase recibe un 
parámetro obligatorio y dos opcionales:

- **classifier_file**: el archivo XML necesario para entrenar el clasificador en cascada. Dependiendo del archivo xml el
clasificador detectará unas regiones u otras.
- **scale_factor**: el factor de escala entre las imágenes generadas por el clasificador. Es opcional, si no se 
introduce, su valor por defecto es 1.3.
- **min_neighbors**: el número mínimo de vecinos que debe tener una región candidata para ser válida. Es opcional, si no 
se introduce, su valor por defecto es 5.

En el constructor se crea y entrena un clasificador denominado *classifier* pasando el fichero obtenido al método 
`cv.CascadeClassifier()`. Los otros dos valores se almacenan en la clase y se podrán obtener mediante los métodos 
`get_scale_factor()` y `get_min_neighbors()` y cambiar mediante los métodos `set_scale_factor()` y `set_min_neighbors()`
, respectivamente. Los dos últimos métodos reciben como parámetro el nuevo valor del atributo.

El método `detect()` recibe una lista de imágenes y, de nuevo opcionalmente, el factor de escala y el mínimo número de 
vecinos. Si no se especifica el valor de estos parámetros, lo obtiene de los atributos almacenado en la clase. El método 
devuelve una lista de arrays de *Numpy*, uno por cada imagen de la lista, con las coordenadas $x$ e $y$ de la esquina 
superior derecha, la altura y la anchura de la región, o regiones, detectadas por el clasificador. El array de *Numpy* 
tendrá 4 columnas, una por cada uno de estos datos, y una fila por cada región detectada. 

In [ ]:
import cv2 as cv


class HaarDetector:
    def __init__(self, classifier_file, scale_factor=1.3, min_neighbors=5):
        self.classifier = cv.CascadeClassifier(classifier_file)
        self.scale_factor = scale_factor
        self.min_neighbors = min_neighbors

    def get_scale_factor(self):
        return self.scale_factor

    def set_scale_factor(self, scale_factor):
        self.scale_factor = scale_factor

    def get_min_neighbors(self):
        return self.min_neighbors

    def set_min_neighbors(self, min_neighbors):
        self.min_neighbors = min_neighbors

    def detect(self, images, scale_factor=None, min_neighbors=None):
        if scale_factor is None:
            scale_factor = self.scale_factor
        if min_neighbors is None:
            min_neighbors = self.min_neighbors
        return [self.classifier.detectMultiScale(image, scale_factor, min_neighbors) for image in images]

A continuación se documenta el fichero *localizacion_matricula.py*

El detector de matrículas haar funciona en las imágenes en las que el coche está en posición frontal, pero si el coche está ligeramente girado, puede no funcionar. Por eso hemos implementado otro detector de matrículas, este basado en componentes conexas de una imagen umbralizada para encontrar un rectángulo blanco: la matrícula. Este detector está implementado en el fichero *localizacion_matricula.py*.

El método *buscar_matricula* recibe una imagen en escala de grises y un detector de coches orb junto a un emparejador de descriptores y un objeto FLANN. Este detector orb se corresponde con el desarrollado en la práctica 1, y detecta el coche que se encuentra en la imagen proporcionada.

A partir del punto obtenido como centro del coche, buscamos rectángulos candidatos a matrícula con el método *get_possible_plates()*. Después, buscamos rectángulos candidatos a carácteres dentro de esas posibles matrículas, y supondremos que la verdadera matrícula es la que más carácteres contiene (la variable plate_index guarda el índice de la matrícula elegida). Si hemos encontrado pocos carácteres (menos de 4), volveremos a buscar posibles matrículas pero aplicando ahora un erosionado en la imagen umbralizada para salvar algún pequeño error en la búsqueda anterior. Cuando encontramos un candidato a matrícula robusto, devolvemos un numpy array con las dimensiones de la matrícula (posición en x, posición en y, anchura y altura).

In [ ]:
def buscar_matricula(image, orb, match_table, flann):
    detected_points = deteccion_orb.detect([image], orb, match_table, flann, 4, 2, 1)
    centre = detected_points[0]
    rect_plates, box_plates = get_possible_plates(image, centre)
    numbers, plate_index = find_numbers_in_plates(image, rect_plates, rotated_plate=True)
    for j in range(2, 5):
        if len(numbers) < 4:
            rect_plates, box_plates = get_possible_plates(image, centre, erode=True, esize=j)
            numbers, plate_index = find_numbers_in_plates(image, rect_plates, rotated_plate=True)
    rect_plate = rect_plates[plate_index]
    return rect_plate

El método *get_possible_plates()* recibe una imagen en escala de grises y la posición del coche. También se puede elegir si se va a realizar el erosionado y con qué tamaño de kernel.

In [ ]:
def get_possible_plates(img, car_position, erode=False, esize=3):
    threshold = umbralizado(img,blur=True, tipo=3, ksize=11, c=2)
    plates = []
    rect_plates, box_plates = find_plate_contours(threshold, car_position)

    if len(plates) == 0 or erode:
        kernel = np.ones((esize, esize), np.uint8)
        eroded_img = cv2.erode(threshold, kernel, iterations=1)
        rect_plates, box_plates = find_plate_contours(eroded_img, car_position)

    return rect_plates, box_plates

El método *find_plate_contours()* recibe una imagen binaria y la posición del coche en la imagen, busca rectángulos que puedan corresponderse con una matrícula y devuelve cada uno de ellos de dos formas, como el rectángulo con ejes paralelos a los de la imagen y como el rectángulo mínimo que contiene la matrícula. 

In [ ]:
def find_plate_contours(binary_img, car_position):
    contours, _ = cv2.findContours(binary_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    plate_rect = []
    plate_box = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 9:
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            x, y, w, h = get_box_size(rect)
            x_p, y_p, w_p, h_p = cv2.boundingRect(cnt)
            if possible_plate(car_position, (x, y, w, h)):
                plate_rect.append([x_p,y_p,w_p,h_p])
                plate_box.append(box)

    return plate_rect, plate_box

Los siguientes métodos nos permiten seleccionar un rectángulo como matrícula siempre que esté a una distancia no muy lejana del centro del coche y la relación de aspecto del rectángulo mínimo sea más o menos la de una matrícula española.

In [ ]:
def possible_plate(car_pos, rect_dim):
    return possible_plate_position(car_pos[0], car_pos[1], rect_dim[0], rect_dim[1]+rect_dim[3]/2) \
           and minimum_plate_size(rect_dim[2], rect_dim[3])

def possible_plate_position(car_x, car_y, rect_x, rect_y):
    return -150 < (car_x - rect_x) < 100 and -50 < (rect_y - car_y) < 200

def minimum_plate_size(width, height):
    return width > 40 and height > 9 and 4.2 < width/height < 5.2

Por último, el método *find_numbers_in_plates()* recibe una imagen en escala de grises, una lista con las posibles matrículas encontradas en esta imagen y un booleano que indica si la matrícula es paralela a los ejes de la imagen o no. Esto nos sirve para determinar el ratio anchura/altura de los carácteres ya que al estar la matrícula rotada los carácteres serán más alargados.

Por cada candidato a matrícula buscamos carácteres dentro y elegimos el candidato que más carácteres tenga. Devolvemos una lista con los carácteres encontrados en ese candidato y el índice del candidato en el array de matrículas para esta imagen.

Para buscar los carácteres utilizamos en método *findContours()* de openCV sobre la imagen de la matrícula recortada y umbralizada.

In [ ]:
def find_numbers_in_plates(img, possible_plates, rotated_plate=False):
    possible_plates = np.array(possible_plates)
    total_chars = []
    chars_found = []
    min_wh_ratio = 1.5 if rotated_plate else 1
    max_wh_ratio = 6 if rotated_plate else 5

    for plate in possible_plates:
        chars_in_plate = []
        plate_img = img[plate[1]:plate[1] + plate[3], plate[0]:plate[0] + plate[2]]
        thresh_img = umbralizado(plate_img, blur=True)
        contours, _ = cv2.findContours(thresh_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 15:
                x, y, w, h = cv2.boundingRect(cnt)
                if min_wh_ratio < h/w < max_wh_ratio:
                    chars_in_plate.append((x,y,w,h))
        chars_in_plate = get_external_rectangles(chars_in_plate)

        chars_found.append(len(chars_in_plate))
        total_chars.append(chars_in_plate)

    if len(chars_found) > 0:
        most_chars_plate_index = np.argmax(chars_found)
        chars_in_plate = total_chars[most_chars_plate_index]

    else:
        chars_in_plate = []
        most_chars_plate_index = 0


    return chars_in_plate, most_chars_plate_index

El fichero *localizacion_caracteres.py* está dedicado a la localización de los caracteres de las matrículas de los 
coches.

El método *load()* recibe como parámetros el directorio del que cargar las imágenes, una lista con los caracteres a 
excluir y una variable que especifica si las 
imágenes deben ser cargadas a color o en niveles de grises. Calcula el directorio actual por si este es relativo y, en 
caso de que lo sea, crea la ruta absoluta. Busca todos los ficheros que se encuentran en el directorio especificado e 
inserta en una lista aquellos que deben incluirse. La lista se ordena y se devuelve otra lista conteniendo las imágenes 
cargadas.

In [ ]:
def load(directory, color=False, exclude=None):
    """Devuelve una lista con las imagenes en color o escala de grises contenidas en el directorio pasado como argumento
    descartando aquellas imagenes cuya primera letra este en la lista de excluidos"""
    if exclude is None:
        exclude = ['.']
    cur_dir = os.path.abspath(os.curdir)
    if directory.startswith("/"):
        path = directory
    else:
        path = cur_dir + '/' + directory
    with os.scandir(path) as it:
        files = [file.name for file in it if file.name[0] not in exclude and file.is_file()]
    it.close()
    files.sort()
    if color is True:
        return [cv.imread(directory + '/' + file) for file in files], files
    else:
        return [cv.imread(directory + '/' + file, 0) for file in files], files

La función *umbralizado()* recibe una lista de imágenes y el tipo de umbralizado. El tipo 0 es el umbralizado 
adaptativo y este requiere dos parámetros especiales, pasados también como parámetro de la función *umbralizado()*. El 
tipo 1 es el umbralizado binario y, el tipo 2, el de Otsu. Además, se puede decidir si se quiere aplicar un 
suavizado a la imagen antes de umbralizar. Se aplica el umbralizado especificado a cada imagen de la lista y se 
devuelven umbralizadas dentro de una lista. 

In [ ]:
def umbralizado(images, blur=False, tipo=0, ksize=5, c=2):
    """Devuelve una lista de imagenes umbralizadas mediante el tipo de umbralizado especificado en los parámetros"""
    imagenes_umbralizadas = []
    for i in range(len(images)):
        image = images[i]

        if blur is True:
            image = cv.GaussianBlur(image, (3, 7), 0)

        if tipo == 0:
            th = cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, ksize, c)
        elif tipo == 1:
            _, th = cv.threshold(image, 127, 255, cv.THRESH_BINARY)
        else:
            _, th = cv.threshold(image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

        imagenes_umbralizadas.append(th)

    return imagenes_umbralizadas

la función *negativo()* recibe una lista de listas con imágenes umbralizadas y devuelve estas imágenes invertidas, 
conservando la estructura de listas anidadas. 

In [ ]:
def negativo(images):
    images_negativo = []
    for image_list in images:
        aux = []
        for image in image_list:
            aux.append(cv.bitwise_not(image))
        images_negativo.append(aux)

    return images_negativo

El siguiente método hace uso de la clase *HaarDetector* para localizar las coordenadas en las que se encuentran las 
matrículas de cada imagen, devolviendo una lista de Numpy arrays con 4 columnas (coordenada x, y, anchura y altura) y 
con tantas filas como matriculas haya detectado.

In [1]:
def get_contorno_matricula_haar(input_images):
    """Devuelve una lista con la posion de la matricula"""
    clasificador_matriculas = haardet.HaarDetector('haar_opencv_4.1-4.2/matriculas.xml')
    return clasificador_matriculas.detect(input_images, 1.1, 5)

La función principal del método, *localizar()*, se encarga de la localización de los caracteres haciendo uso de los 
métodos antes descritos, además de otros implementados en otros ficheros.

Comienza cargando todas las imágenes del directorio en niveles de gris y en blanco y negro. Las imágenes en niveles de 
gris se utilizarán para la detección de caracteres mediante el clasificador haar de la clase *HaarDetector*. Las 
coordenadas de matrículas detectadas se almacenan en la variable *matriculas*. Sin embargo, el detector haar no detecta 
todas las matrículas, no es capaz de detectar las que no están de frente a la cámara. Es por ello que se recorre la 
lista de matrículas detectadas y se vuelve a detectar para las imágenes en las que el haar no fue útil. Esta segunda 
detección se hace mediante un detector ORB.

In [ ]:
def localizar(directory):
    # Imagenes sobre las que detectar en escala de grises. Por cada imagen hay una lista con un elemento por cada
    # matricula detectada
    input_images, _ = load(directory)
    # Lo mismo en color
    input_images_color, _ = load(directory, color=True)
    # Lista de coordenadas de las matriculas. Cada array tiene tantas filas como matriculas hay en la imagen
    matriculas = get_contorno_matricula_haar(input_images)

    train_images = deteccion_orb.load()
    orb = cv.ORB_create(nfeatures=300, scaleFactor=1.3, nlevels=4)
    match_table, flann = deteccion_orb.train(train_images, orb)

    matriculas_total = []
    caracteres = []
    
    for i in range(len(matriculas)):
        if len(matriculas[i]) > 0:
            numbers, _ = localizacion_matricula.find_numbers_in_plates(input_images[i], matriculas[i])
            rect_plate = matriculas[i]
            rect_plate = rect_plate[0]
            caracteres.append([numbers])
            matriculas_total.append([rect_plate])
        else:
            detected_points = deteccion_orb.detect([input_images[i]], orb, match_table, flann, 4, 2, 1)
            centre = detected_points[0]
            rect_plates, box_plates = localizacion_matricula.get_possible_plates(input_images[i], centre)
            numbers = localizacion_matricula.find_numbers_in_plates(input_images[i], rect_plates, rotated_plate=True)
            for j in range(2, 5):
                if len(numbers) < 4:
                    rect_plates, box_plates = localizacion_matricula.get_possible_plates(input_images[i], centre,
                                                                                         erode=True, esize=j)
                    numbers, plate_index = localizacion_matricula.find_numbers_in_plates(input_images[i], rect_plates,
                                                                                         rotated_plate=True)
            if len(rect_plates) > 0:
                rect_plate = rect_plates[plate_index]
            caracteres.append([numbers])
            matriculas_total.append(np.array([rect_plate]))

Una vez que todas las matrículas están detectadas y almacenadas en la variable *matriculas_total*, se recorren esta 
lista y la que contiene las imágenes originales para recortar la región que pertenece a la matrícula. Este recorte no 
se realiza en el momento de la localización de los caracteres porque más tarde necesitaremos las coordenadas de estos. 
Para las matrículas con 8 o más caracteres detectados, cosas que no puede ocurrir, se seleccionan las 7 que 
corresponden a caracteres en función de la distancia de la altura de los caracteres con respecto a la media de las 
alturas.

La aplicación se ha desarrollado de modo que se permite la detección de varias matrículas por imagen, es por ello que 
las estructuras de datos contienen listas anidadas. Cada lista tendrá una lista en su interior por cada imagen, y cada 
una de estas, a su vez, contendrá una lista por cada matrícula detectada, y estas últimas albergarán en ellas las 
coordenadas de los caracteres de dicha matrícula. También es por esta razón que haya métodos como *get_contornos()* o 
*get_contornos_lista()*. La primera función está enfocada a la detección de una sola matrícula por imágen, mientras 
que la segunda soporta varias y hace uso de la primera.

Por último, los caracteres son recortados de las imágenes originales, umbralizados e invertidos. La función devuelve 
estas últimas imágenes con los caracteres recortados y las coordenadas de las matrículas y dichos caracteres.

In [ ]:
# Lista de regiones de las imagenes conteniendo las matriculas
    roi_matricula = []
    for (img, mat) in zip(input_images, matriculas_total):
        aux = []
        for (x, y, w, h) in mat:
            aux.append(img[y:y + h, x:x + w])
        roi_matricula.append(aux)

    # De todos los caracteres queremos obtener 7, ya que tambien detecta la E y las sobras de izquierda y derecha
    caracteristicas_seleccionadas = []
    for coche in caracteres:
        aux_coche = []
        for matricula in coche:
            aux_matricula = []
            medias = []
            for (x, y, w, h) in matricula:
                medias.append((h, (x, y, w, h)))

            if (len(medias) < 8):
                for (_, (x, y, w, h)) in medias:
                    aux_matricula.append((x, y, w, h))
            else:
                mean_y = 0
                for (y, _) in medias:
                    mean_y += y
                mean_y = mean_y / len(medias)

                medias.sort(key=lambda r: np.abs(r[0] - mean_y))
                for (_, (x, y, w, h)) in medias[:7]:
                    aux_matricula.append((x, y, w, h))

            aux_matricula.sort(key=coordenada_x)
            aux_coche.append(aux_matricula)

        caracteristicas_seleccionadas.append(aux_coche)

    # Obtiene las secciones de imagen con los caracteres
    roi_caracteres = []
    for (image_list, coche) in zip(roi_matricula, caracteristicas_seleccionadas):
        aux = []
        for (image, mat) in zip(image_list, coche):
            aux.append([image[y:y + h, x:x + w] for (x, y, w, h) in mat])
        roi_caracteres.append(aux)

    # Las vuelve a unmbralizar e invertir
    roi_caracteres_umbral = []
    for li in range(len(roi_caracteres)):
        roi_caracteres_umbral.append(umbralizado_lista(roi_caracteres[li], False, 2))

    roi_caracteres_umbral_inv = []
    for li in roi_caracteres_umbral:
        roi_caracteres_umbral_inv.append(negativo(li))

    return roi_caracteres_umbral_inv, matriculas_total, caracteristicas_seleccionadas

# Apartado 2: Reconocimiento de los caracteres localizados

Este apartado aparece desarrollado en el fichero *reconocimiento_caracteres.py*. Al principio de este se importan las 
librerías a utilizar y el fichero *localizacion_caracteres* desarollado en el apartado anterior.

`os` permite la obtención de los archivos que se encuentran en un directorio para ser cargados posteriormente mediante 
`cv2`; `numpy` servirá para la creación de matrices multidimensionales; y las clases `LinearDiscriminantAnalysis` y 
`GaussianNB` de `sklearn` permitirán la reducción de la dimensionalidad de las matrices y la clasificación de los 
caracteres localizados, respectivamente. 

In [ ]:
import os
import numpy as np
import cv2 as cv
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
import localizacion_caracteres as localizacion

A continuación, se definen los siguientes métodos:

El método `get_chars_countour()`recibe una lista de imágenes como parámetro. Para cada imagen, localiza las compomentes 
conexas mediante la llamada a la función `cv.findContours()`. Esta función recibe como parámetro:

- **image**: la imagen umbralizada sobre la que localizar las componentes conexas. La imagen debe tener un solo canal 
y los contornos a detectar deben tener un valor distinto de 0, ya que los 0s se toman como fondo y los mayores 
que 0 (tomados como 1s), son los que se detectan. 
- **mode**: especifica los contornos a devolver. Se necesitan todos los contornos sin establecer relaciones de jerarquía 
, por lo que se pasa `cv.RETR_LIST`.
- **method**: especifica el método de aproximación de los contornos. Ya que se busca obtener todos los puntos sin 
aproximación, se pasa `cv.CHAIN_APPROX_NONE`.

La función devuelve los contornos de las componentes conexas encontradas en la imagen (variable *contour*) y la 
jerarquía de los puntos. Ya que no se han establecido relaciones, no es necesario almacenar esta información. Para cada 
componente conexa encontrada, se obtiene el recuadro que rodea el contorno de esta llamando a la función 
`cv.boundingRect()` y pasando como argumento el contorno. Para cada imagen se obtendrá una lista con el rectángulo que 
rodea la región de interés, y esta lista se almacenará en otra con el resto de recuadros del resto de imágenes. La lista 
devuelta contendrá, a su vez, una lista por cada imagen, conteniendo los recuadros de los contornos de cada imagen.

In [ ]:
def get_chars_countour(images):
    """Devuelve una lista con tantas listas como imagenes, cada una de estas conteniendo los caracteres encontrados
    en cada imagen"""
    char_contour = []
    for image in images:
        aux = []
        contour, _ = cv.findContours(image, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
        for cnt in contour:
            x, y, w, h = cv.boundingRect(cnt)
            aux.append((x, y, w, h))
        char_contour.append(aux)

    return char_contour

La siguiente función, *get_chars_image()* recibe una lista de imágenes y las coordenadas de la posicion de los 
contornos detectados en estas y devuelve una lista de imágenes con la region del contorno recortado.

- **images**: lista de imágenes.
- **characters**: lista con tantas listas como imágenes, donde cada una de estas contiene la posición de los 
recuadros que contienen los contornos detectados.
- **cont**: array de *Numpy* con una fila y tantas columas como posibles clases se pueden detectar.

Se empareja cada imagen y su lista de contornos detectados correspondiente y se ordenan los recuadros por área 
descendiente. Esto se debe a que a veces se detectan dos contornos, uno con la región deseada, y otro con una sección 
interna de este que no interesa. Se obtiene el primer contorno, el que engloba toda la región de interés y se añade a 
la lista. Además, se incrementa el contador de la clase del contorno detectado. Devuelve una lista con el recorte de la 
imagen original donde aparece la componente conexa localizada. 

In [ ]:
def get_chars_image(images, characters, cont):
    """Devuelve una lista con los caracteres detectados en la lista, y un contador que indica cuantas muestras hay de
    cada letra o numero"""
    chars_images = []
    for i in range(len(images)):
        image = images[i]
        chars = characters[i]
        chars.sort(key=area, reverse=True)
        if len(chars) > 0:
            (x, y, w, h) = chars[0]
            chars_images.append(image[y:y + h, x:x + w])
            cont[i // 250] += 1

    return chars_images, cont

Una última función `area()` recibe un contorno y devuelve el área de este.

In [ ]:
def area(contour):
    """Devuelve el area de un rectangulo"""
    return contour[2] * contour[3]

A continuación, se define la clase `ReconocimientoCaracteres`. El constructor recibe lo siguiente:

- **included_characters**: lista con los caracteres con los que se va a entrenar el clasificador.
- **directory**: directorio en el que se encuentran las imágenes que se van a usar para entrenar el clasificador.

Almacena los valores recibidos y crea un *LDA* y un clasificador de Bayes gausiano.

EL método *__load()* es privado y su función es cargar en una lista las imágenes que corresponden a los caracteres 
contenidos en *included_characters*. El nombre de los ficheros de imagen empieza con el caracter en cuestión, por lo 
que, de los archivos obtenidos mediante `os.scandir()`, solamente se incluyen los archivos cuyo nombre empieza por
alguna de las letras incluidas en la lista. Se obtiene una lista con las rutas de las imágenes y se ordenan. El 
método recibe dos argumentos opcionales, un *boolean* que especifica si se desea la imagen en color o no y otro que 
especifica si se deben invertir las imágenes cargadas. Para la carga de las imágenes se emplea el método `cv.imread()`, 
que recibe la ruta de la imagen a leer y, opcionalmente, el índice del canal a cargar (se cargan todos por omisión). 
Para invertir las imágenes se usa la función `cv.bitwise()`, que recibe una imagen e intercambia el valor del píxel por 
el valor de restar a 255 el valor actual. De este modo, los contornos de interés se procesarán mas adelante como 1s.

El método *train()* corresponde a la fase de entrenamiento del LDA y el GNB. Primero carga las imágenes mediante 
 ` __load()`y a continuación le aplica el umbralizado con el método `threshold()` del módulo `localizacion`.
  En el umbralizado no aplica difuminado sobre la imagen y emplea el método de Otsu. La lista 
obtenida contiene las imágenes con dos valores de píxel, 0 y 255. Ya que se aplicó un filtro de negativo a las imágenes 
en la carga, los caracteres estarán a 255 y, el fondo, a 0. Las imágenes obtenidas se pasan a `get_chars_contour()` 
para encontrar las coordenadas de los recuadros que abarcan el contorno de los caracteres. La variable `cont` es una 
matriz de ceros con una fila y tantas columnas como posibles caracteres hay que detectar. El método `get_chars_image()` 
recibe la lista inicial de imágenes cargadas, las coordenadas de los caracteres localizados en estas imágenes y esta 
última variable `cont`, que servirá para contar cuántas muestras hay de cada caracter. La función devuelve una lista 
con tantas listas como imágenes hay, cada lista de estas conteniendo un recorte de la imagen original conteniendo los 
caracteres detectados; y el contador actualizado. Estas imágenes con los caracteres son sometidas a un cambio de tamaño. 
De esto se encarga el método `cv.resize()`. Este recibe como argumento cada una de las imágenes de los caracteres, el 
tamaño final deseado, en este caso una tupla (10, 10) para que pase a tener un tamaño de 10x10 píxeles; y el método de 
interpolación, en este caso `cv.INTER_LINEAR` para realizar una interpolación bilineal. Reorganizando las filas de la 
matriz que representa la imagen en una fila de 100 columnas y disponiendo cada imagen de entrenamiento como una fila de 
la matriz se obtiene la matriz C, el vector de características. De la variable `cont` se obtiene la matriz E, que 
indica a qué clase pertenece cada 
fila de la matriz, es decir, cada imagen. Con estas dos matrices y el método `fit()` se entrena el LDA y mediante 
`transform()` se reduce el vector de características. Y es con este último el que sirve para entrenar finalmente el 
clasificador bayesiano.

Por último, el método `detect()` recibe una matriz D similar a C y que será el vector de características de los 
caracteres a reconocer. Se reduce la dimensionalidad de la matriz D mediante en LDA del mismo modo y se devuelve una 
matriz con tantas columnas como elementos reconocidos, conteniendo cada una de estas el índice de la matriz de 
caracteres incluidos que corresponde al caracter reconocido. 

In [ ]:
class ReconocimientoCaracteres:
    """Reconocedor de caracteres en matriculas mediante LDA y GNB"""

    def __init__(self, included_characters, directory):
        """Crea un LDA y un GNB para el reconocimiento de caracteres"""
        self.included_characters = included_characters
        self.images_directory = directory
        self.lda = LinearDiscriminantAnalysis()
        self.gnb = GaussianNB()

    def __load(self, color=False, invert=True):
        """Devuelve una lista con las imagenes contenidas en el directorio de imagenes"""
        cur_dir = os.path.abspath(os.curdir)
        with os.scandir(cur_dir + '/' + self.images_directory) as it:
            files = [file.name for file in it if file.name[0] in self.included_characters and file.is_file()]
        it.close()
        files.sort()
        if color is True:
            return [cv.imread(self.images_directory + '/' + file) for file in files]
        else:
            if invert is True:
                return [cv.bitwise_not(cv.imread(self.images_directory + '/' + file, 0)) for file in files]
            else:
                return [cv.imread(self.images_directory + '/' + file, 0) for file in files]

    def train(self):
        """Entrena un clasificador Naive Bayes para la deteccion de caracteres"""
        input_images = self.__load()
        input_images_threshold = localizacion.threshold(input_images, False, 2)
        input_chars = get_chars_countour(input_images_threshold)

        cont = np.zeros((len(self.included_characters)), dtype=np.uint16)
        roi_input_chars, cont = get_chars_image(input_images, input_chars, cont)

        roi_chars_resized = [cv.resize(image, (10, 10), 0, 0, cv.INTER_LINEAR) for image in roi_input_chars]

        C = np.array([char.reshape(1, 100).astype(np.float64) for char in roi_chars_resized])[:, 0, :]
        E = [i for i in range(cont.shape[0]) for _ in range(cont[i])]

        self.lda.fit(C, E)
        CR = self.lda.transform(C)
        self.gnb.fit(CR, E)

        return C, E

    def detect(self, D):
        """Reconoce los caracteres de la matriz D"""
        DR = self.lda.transform(D)
        return self.gnb.predict(DR)

# Apartado 3: Integración de todos los algoritmos

En el fichero *leer_coche.py* realizamos algunas operaciones básicas como cargar las imágenes, controlar la llamada
al programa, mostrar las imágenes finales si es requerido y crear el fichero de texto con las matrículas reconocidas.

La función *get_name()* elimina la parte de la ruta que no es el fichero

In [ ]:
def get_name(path):
    if path.find('/') != -1:
        filename = path[path.find('/'):]
    elif path.find('\\') != -1:
        filename = path[path.find('\\'):]
    else:
        filename = path
    return filename

Casi todo esto se integra en el método *leer()*, así que vamos a verlo por partes.

Primero cargamos las imágenes en color y almacenamos los nombres de las imágenes. Tras esto abrimos un fichero de texto
 en el que se volcarán las matrículas detectadas y creamos los conjuntos de letras y números cuya unión resulta en el
 conjunto de carácteres posibles en una matrícula española.

Al final de este fragmento inicializamos y entrenamos dos clasificadores, uno de números y otro de letras consonantes.
 Esto nos sirve para minimizar posibles errores al clasificar por ejemplo una __D__ como un **0**.

In [ ]:
def leer(directorio, visualizar):
    input_images_color, nombre_imagenes = localizacion.load(directorio, color=True)

    filename = get_name(directorio) + '.txt'
    file = open(filename, 'w')

    numbers = [str(n) for n in list(range(10))]
    letters = string.ascii_uppercase
    valid_letters = [c for c in
                     letters.replace('A', '').replace('E', '').replace('I', '').replace('O', '').replace('U', '')]

    numeros = ReconocimientoCaracteres(numbers, 'training_ocr')
    C, E = numeros.entrenar()
    # numeros.test(C)

    letras = ReconocimientoCaracteres(valid_letters, 'training_ocr')
    C, E = letras.entrenar()
    # letras.test(C)

Ahora hacemos una llamada al método que busca las matrículas y los carácteres que se encuentran dentro de ellas sobre todas las imágenes del directorio especificado. En la variable *local* se guarda un array por cada imagen que contiene un array de arrays con los carácteres encontrados en cada matrícula de esa imagen. Para cada uno de estos carácteres vamos a redimensionar la imagen que lo contiene y cambiar la forma del array para obtener un vector con 100 características.

In [ ]:
local, matriculas, caracteres = localizacion.localizar(directorio)

    local_resized = []
    for image_list in local:
        aux = []
        for image in image_list:
            aux.append(
                [cv.resize(char, (10, 10), 0, 0, cv.INTER_LINEAR).reshape(1, 100).astype(np.float64) for char in image])
        local_resized.append(aux)

Como último paso, se recorren los caracteres y las matrículas localizadas, construyendo la matriz F, que corresponde
al vector de características de los caracteres a detectar. Se dispone de un clasificador para letras y otra para
números, y nos basamos en el indice de la posición del caracter para clasificar mediente uno u otro.
Datos como la posición de la matrícula y los caracteres encontrados se almacenan en un fichero y, si se ha pasado el
parámetro 'True' en la llamada al programa, se muestran las imágenes por pantalla con los caracteres detectados.

In [ ]:
    for img in range(len(local_resized)):
        imagen = input_images_color[img]
        for m in range(len(local_resized[img])):
            if len(local_resized[img][0]) > 0:
                F = np.array([char for char in local_resized[img][m]])[:, 0, :]
                out_numeros = numeros.reconocer(F[:4])
                out_letras = [letras.included_characters[i] for i in letras.reconocer(F[-3:])]
                texto_matricula_numeros = str(out_numeros[0]) + str(out_numeros[1]) + str(out_numeros[2]) \
                                          + str(out_numeros[3])
                texto_matricula_letras = str(out_letras[0]) + str(out_letras[1]) + str(out_letras[2])

                (x, y, w, h) = matriculas[img][m]

                nombre_imagen = get_name(nombre_imagenes[img])
                x_centro_matricula = x + w // 2
                y_centro_matricula = y + h // 2
                texto_matricula = texto_matricula_numeros + ' ' + texto_matricula_letras
                mitad_largo_matricula = w // 2
                file.write(str(nombre_imagen) + ' ' + str(x_centro_matricula) + ' ' + str(y_centro_matricula) + ' ' +
                           texto_matricula + ' ' + str(mitad_largo_matricula) + '\n')

                imagen = cv.rectangle(imagen, (x, y), (x + w, y + h), (0, 255, 0), 2)
                imagen = cv.circle(imagen, (x_centro_matricula, y_centro_matricula), 5, (0, 255, 0), thickness=2,
                                   lineType=8, shift=0)

                matricula = caracteres[img][m]
                caracteres_matricula = texto_matricula.replace(' ', '')
                for i in range(len(matricula)):
                    (a, b, c, d) = matricula[i]
                    imagen = cv.rectangle(imagen, (x+a, y+b), (x+a + c, y+b + d), (0, 0, 255), 1)
                    imagen = cv.putText(imagen, caracteres_matricula[i], (x+a,y+b), cv.FONT_HERSHEY_SIMPLEX,
                                    1, (255, 0, 0), 1, cv.LINE_AA)

        if visualizar:
            plt.imshow(cv.cvtColor(imagen, cv.COLOR_RGB2BGR))
            plt.show()
            pass

    file.close()

Este último fragmento corresponde a la ejecución de todos los métodos cuando se invoca la aplicación en una terminal.

In [ ]:
if __name__ == "__main__":
    assert len(sys.argv) > 1, \
        "Debes introducir al menos un argumento, el directorio donde se encuentran las imagenes.\n" \
        "Uso: python leer_coche.py <path_completo_directorio_coches> [<visualizar_resultados>] \n" \
        "El parametro entre corchetes es opcional y su valor por defecto es 'False'."

    if len(sys.argv) > 2:
        visualizar_resultados = sys.argv[2]
    else:
        visualizar_resultados = False
    directorio_imagenes = sys.argv[1]

    leer(directorio_imagenes, visualizar_resultados)

# Apartado 4: Ejecución

La práctica deberá ejecutarse sobre Python 3.7.X y OpenCV 4.3 mediante el siguiente mandato:

<code>>$ python leer_coche.py [directorio de imágenes de testing] [visualización de imágenes por pantalla]</code>
donde los argumentos son los siguientes:

-   *directorio de imágenes de testing*: Ruta relativa del directorio que contiene las imágenes de testing.

-   *visualización de imágenes por pantalla*: Puede tomar los valores *True* o *False*.
